#**Extraindo e Analisando Notícias da Infonet**
O presente notebook foi criado no intuito de auxiliar em uma tarefa de Web Scraping no âmbito do Laboratório Multiusuário de Informática e Documentação Linguística (LAMID/UFS).</br>
A tarefa em questão foi a coleta e análise de dados contidos nas notícias do site [Infonet](https://infonet.com.br/).</br>
> Os dados extraídos foram:
* Data da publicação;
* Título;
* Resumo;
* Autoria;
* Link;
* Texto da Notícia.


###**Montando o drive**
O notebook foi criado no ambiente do Google Colab, então para salvarmos os arquivos de saída, conectamos o ambiente de programação ao Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##**Extraindo os dados**

###**Importando as bibliotecas necessárias**

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

###**Criando funções de busca**
Aqui são definidas as funções para a extração de cada dado de interesse:

*   **extraiData**: data de publicação da notícia;
*   **extraiTitulo**: título da notícia;
*   **extraiResumo**: resumo da notícia;
*   **extraiAutoria**: autoria registrada no código fonte da notícia;
*   **extraiLink**: link para a notícia completa;
*   **extraiTexto**: texto completo da notícia.



In [ ]:
def extraiData(noticia):
    data = noticia.find('span',{'class':'time'})
    return data.get_text()

def extraiTitulo(noticia):
    titulo = noticia.find('a', {'class':'post-url post-title'})
    return titulo.get_text().strip()

def extraiResumo(noticia):
    resumo = noticia.find('div', {'class':'post-summary'})
    return resumo.get_text().strip()

def extraiAutoria(noticia):
    autoria = noticia.find('i', {'class':'post-author author'})
    return autoria.get_text().strip()

def extraiLink(noticia):
    link = noticia.find('h2').find('a').attrs['href']
    return link

def extraiTexto(link):
    html = urlopen(link)
    bs = BeautifulSoup(html.read(), 'html.parser')
    texto = bs.find('div', {'class':'entry-content clearfix single-post-content'})
    return texto.get_text().strip()

###**Extraindo e armazenando os dados**
Aqui vamos inserir o link da página que desejamos extrair os dados na variável `LINK`. </br>
> **Observação**: é importante lembrar que esse código foi desenvolvido para funcionar com links do site da [infonet](https://infonet.com.br/). Então, é necessário que seja realizada uma busca por algum termo específico na parte de "pesquisa" do site e que o link exibido seja copiado e colado na variável especificada (`LINK`).

Além disso, iremos inserir o número de páginas que desejamos coletar as notícias (no site da Infonet são exibidas **10 notícias por página**). O código irá abrir o número de páginas definido na variável `NUMERO_DE_PAGINAS`, e executar as funções para extrair os elementos que precisamos.


In [ ]:
#Link de busca
LINK = 'https://infonet.com.br/'

NUMERO_DE_PAGINAS = 10

In [ ]:
colunas = ['Link', 'Data', 'Título', 'Resumo', 'Autor(a)', 'Texto da Notícia']
manchetes = []

for i in range(1, NUMERO_DE_PAGINAS + 1):
    link = LINK.replace('https://infonet.com.br/', f'https://infonet.com.br/page/{i}/')

    html = urlopen(link)
    bs = BeautifulSoup(html.read(), 'html.parser')

    noticias = bs.findAll('article')

    for noticia in noticias:
        link = extraiLink(noticia)
        manchetes.append([link, extraiData(noticia), extraiTitulo(noticia), extraiResumo(noticia), extraiAutoria(noticia), extraiTexto(link)])

###**Criando data frame**
Aqui, passamos como parâmetros os nomes das colunas que desejamos ver em nossa planilha (definidos na lista `colunas` na célula anterior) e os dados, que foram armazenados na lista `manchetes`.</br>Passados os parâmetros, o código irá organizar os dados em uma estrutura chamada _Data frame_.
</br>
> As linhas `df.head()` e `df.info()` exibem as primeiras linhas do dataframe e algumas informações, como o número de dados armazenados. Essas funções são interessantes para sabermos se os dados foram armazenados corretamente.

In [ ]:
df = pd.DataFrame(manchetes, columns = colunas)

df.head()
df.info()

###**Salvando a planilha com os dados**
> **Planilha sem análise**</br>


Por fim, é interessante que os dados sejam salvos. Aqui iremos inserir o nome que queremos que nosso arquivo tenha na variável `NOME_ARQUIVO_DADOS` e rodar a célula para que a planilha seja salva.

In [ ]:
NOME_ARQUIVO_DADOS = 'Dados Infonet'

#Salvar em .xlsx (Excel)
planilhaDados = df.to_excel(f'{NOME_ARQUIVO_DADOS}.xlsx', encoding='UTF-8')

#Salvar em .csv
#planilhaDados = df.to_csv(f'{NOME_ARQUIVO_DADOS}.csv', encoding='UTF-8')

##**Analisando os dados**

###**Instalando e importando as bibliotecas necessárias**

In [ ]:
!pip install spacy
!python -m spacy download pt_core_news_lg
import spacy
import pandas as pd

###**Analisando a voz verbal nos títulos**
Definida a função `vozVerbal`, iremos aplicá-la nos títulos coletados. Como os dados já foram extraidos e estão no dataframe, podemos acessá-los inserindo apenas o nome da sua coluna correspondente, como foi feito chamando a coluna `df['Título']`.

Em seguida, criamos uma nova coluna chamada **Análise de Voz Verbal** que aramzenará os títulos contendo as análises.

In [ ]:
titulosAnalisados = []

nlp = spacy.blank("pt")
nlp = spacy.load("pt_core_news_lg")

for titulo in df['Título']:
    doc = nlp(titulo)

    analiseVozVerbal = ''
    for token in doc:
        if str(token.pos_) == 'VERB':
            if 'Voice=Pass' in str(token.morph):
                analiseVozVerbal += ' ' + token.text + ' (Voz PASSIVA)'
            else:
                analiseVozVerbal += ' ' + token.text + ' (Voz ATIVA)'
        else:
            analiseVozVerbal += ' ' + token.text

    titulosAnalisados.append(analiseVozVerbal)

df['Análise de Voz Verbal'] = titulosAnalisados

df.head()

###**Salvando a planilha com dados e análises**
> **Com análise**


A planilha seguir possui o acréscimo de uma última coluna contendo a análise de voz verbal dos títulos.


In [ ]:
NOME_ARQUIVO_ANALISE = '(Análise) Dados Infonet'

#Salvar em .xlsx (Excel)
planilhaDados = df.to_excel(f'{NOME_ARQUIVO_ANALISE}.xlsx', encoding='UTF-8')

#Salvar em .csv
#planilhaDados = df.to_csv(f'{NOME_ARQUIVO_ANALISE}.csv', encoding='UTF-8')